In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data

# Load dataset

In [2]:
dataset = dgl.data.CoraGraphDataset()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
g = dataset[0]

In [5]:
g.ndata

{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}

In [6]:
g.edata

{}

In [15]:
dataset.num_classes

7

# Graph model

In [10]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

hdim = 16
nclasses = dataset.num_classes
# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], hdim, nclasses)

In [9]:
g.ndata['feat'].shape

torch.Size([2708, 1433])

# Training

In [11]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
train(g, model)

/home/chenmin/miniconda3/envs/graph/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 1.946, val acc: 0.100 (best 0.100), test acc: 0.083 (best 0.083)
In epoch 5, loss: 1.890, val acc: 0.518 (best 0.528), test acc: 0.538 (best 0.536)
In epoch 10, loss: 1.805, val acc: 0.632 (best 0.632), test acc: 0.646 (best 0.646)
In epoch 15, loss: 1.693, val acc: 0.642 (best 0.668), test acc: 0.667 (best 0.668)
In epoch 20, loss: 1.557, val acc: 0.652 (best 0.668), test acc: 0.687 (best 0.668)
In epoch 25, loss: 1.402, val acc: 0.688 (best 0.688), test acc: 0.712 (best 0.712)
In epoch 30, loss: 1.234, val acc: 0.718 (best 0.718), test acc: 0.727 (best 0.727)
In epoch 35, loss: 1.060, val acc: 0.728 (best 0.728), test acc: 0.735 (best 0.735)
In epoch 40, loss: 0.891, val acc: 0.730 (best 0.730), test acc: 0.741 (best 0.741)
In epoch 45, loss: 0.735, val acc: 0.746 (best 0.746), test acc: 0.746 (best 0.746)
In epoch 50, loss: 0.598, val acc: 0.748 (best 0.752), test acc: 0.750 (best 0.749)
In epoch 55, loss: 0.482, val acc: 0.754 (best 0.754), test acc: 0.755 (best 0